In [11]:
import pandas as pd
chunk_size = 10000
batch_no = 1
file_name = "C://Users//Harish//Desktop//malware_train.csv"
for chunk in pd.read_csv(file_name,chunksize=chunk_size):
    chunk.to_csv('malware'+str(batch_no)+'.csv',index=False)
    batch_no+=1

C:\Users\Harish\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


KeyboardInterrupt: 

In [16]:
import warnings
import numpy as np
import pandas as pd
import pickle
import dill
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
# Donot modify the method signatures and return dtypes
# you can include additional functions of your choice

class MalwareDetectionModel:

    def __init__(self):
        self.dtypes = {
            'MachineIdentifier':                                    'category',
            'ProductName':                                          'category',
            'EngineVersion':                                        'category',
            'AppVersion':                                           'category',
            'AvSigVersion':                                         'category',
            'IsBeta':                                               'int8',
            'RtpStateBitfield':                                     'float16',
            'IsSxsPassiveMode':                                     'int8',
            'DefaultBrowsersIdentifier':                            'float32',  # was 'float16'
            'AVProductStatesIdentifier':                            'float32',
            'AVProductsInstalled':                                  'float16',
            'AVProductsEnabled':                                    'float16',
            'HasTpm':                                               'int8',
            'CountryIdentifier':                                    'int16',
            'CityIdentifier':                                       'float32',
            'OrganizationIdentifier':                               'float16',
            'GeoNameIdentifier':                                    'float16',
            'LocaleEnglishNameIdentifier':                          'int16',  # was 'int8'
            'Platform':                                             'category',
            'Processor':                                            'category',
            'OsVer':                                                'category',
            'OsBuild':                                              'int16',
            'OsSuite':                                              'int16',
            'OsPlatformSubRelease':                                 'category',
            'OsBuildLab':                                           'category',
            'SkuEdition':                                           'category',
            'IsProtected':                                          'float16',
            'AutoSampleOptIn':                                      'int8',
            'PuaMode':                                              'category',
            'SMode':                                                'float16',
            'IeVerIdentifier':                                      'float16',
            'SmartScreen':                                          'category',
            'Firewall':                                             'float16',
            'UacLuaenable':                                         'float64', # was 'float32'
            'Census_MDC2FormFactor':                                'category',
            'Census_DeviceFamily':                                  'category',
            'Census_OEMNameIdentifier':                             'float32', # was 'float16'
            'Census_OEMModelIdentifier':                            'float32',
            'Census_ProcessorCoreCount':                            'float16',
            'Census_ProcessorManufacturerIdentifier':               'float16',
            'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
            'Census_ProcessorClass':                                'category',
            'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
            'Census_PrimaryDiskTypeName':                           'category',
            'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
            'Census_HasOpticalDiskDrive':                           'int8',
            'Census_TotalPhysicalRAM':                              'float32',
            'Census_ChassisTypeName':                               'category',
            'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
            'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
            'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
            'Census_PowerPlatformRoleName':                         'category',
            'Census_InternalBatteryType':                           'category',
            'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
            'Census_OSVersion':                                     'category',
            'Census_OSArchitecture':                                'category',
            'Census_OSBranch':                                      'category',
            'Census_OSBuildNumber':                                 'int16',
            'Census_OSBuildRevision':                               'int32',
            'Census_OSEdition':                                     'category',
            'Census_OSSkuName':                                     'category',
            'Census_OSInstallTypeName':                             'category',
            'Census_OSInstallLanguageIdentifier':                   'float16',
            'Census_OSUILocaleIdentifier':                          'int16',
            'Census_OSWUAutoUpdateOptionsName':                     'category',
            'Census_IsPortableOperatingSystem':                     'int8',
            'Census_GenuineStateName':                              'category',
            'Census_ActivationChannel':                             'category',
            'Census_IsFlightingInternal':                           'float16',
            'Census_IsFlightsDisabled':                             'float16',
            'Census_FlightRing':                                    'category',
            'Census_ThresholdOptIn':                                'float16',
            'Census_FirmwareManufacturerIdentifier':                'float16',
            'Census_FirmwareVersionIdentifier':                     'float32',
            'Census_IsSecureBootEnabled':                           'int8',
            'Census_IsWIMBootEnabled':                              'float16',
            'Census_IsVirtualDevice':                               'float16',
            'Census_IsTouchEnabled':                                'int8',
            'Census_IsPenCapable':                                  'int8',
            'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
            'Wdft_IsGamer':                                         'float16',
            'Wdft_RegionIdentifier':                                'float16',
            'HasDetections':                                        'int8',
        }
        self.file_name = "malware10.csv"
        self.scaler = StandardScaler()
        self.remove_cols = []
        
    def clean(self, test_data):
        test_data.drop(columns=self.remove_cols, inplace=True)
        # clean up NaN values
        for column in test_data.columns:
            test_data[column].fillna(test_data[column].mode()[0], inplace=True)

        # encode string values
        for col in test_data.columns:
            test_data[col] = self.le.fit_transform(test_data[col])
        return test_data

    def _clean(self, df):
        # determine skewedness
        skew = pd.DataFrame([{'column': c, 'uniq': df[c].nunique(), 'skew': df[c].value_counts(normalize=True).values[0] * 100} for c in df.columns])
        skew = skew.sort_values('skew', ascending=False)
        self.remove_cols.extend(skew[(skew['skew'] > 90)].column.tolist())

        # determine null columns
        total = df.isnull().sum().sort_values(ascending=False)
        percent = (df.isnull().sum()/df.isnull().count()
                   * 100).sort_values(ascending=False)
        null_cols = pd.concat([total, percent], axis=1,
                              keys=['Total', 'Percent'])
        self.remove_cols.extend(null_cols[(null_cols['Percent'] > 5)].index.tolist())

        
        #Remove correlated columns
        corr_matrix = df.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        self.remove_cols.extend([column for column in upper.columns if any(upper[column] > 0.95)])
        self.remove_cols.append('MachineIdentifier')
    
        print(self.remove_cols)
        df.drop(columns=self.remove_cols, inplace=True)

        # clean up NaN values
        for column in df.columns:
            df[column].fillna(df[column].mode()[0], inplace=True)

        # encode string values
        self.le = LabelEncoder()
        for col in df.columns:
            df[col] = self.le.fit_transform(df[col])        
        
       # set index to be the machine IDs
        # df.set_index('MachineIdentifier', inplace=True)
        return df 


    # trains a model, you can access the malware_train.csv file for training your model
    def train(self):
        # read in csv into dataframe
        df = pd.read_csv(self.file_name, dtype=self.dtypes)
        df = self._clean(df)

        # prediction column
        y_train = df['HasDetections']
        # feature columns
        x_train = df.drop('HasDetections', 1)
        x_train[x_train.columns] = self.scaler.fit_transform(x_train[x_train.columns])
        
        #Principal Component Analysis
#         pca = PCA(n_components = None)
#         x_train[x_train.columns] = pca.fit_transform(x_train[x_train.columns])
#         explained_variance = pca.explained_variance_ratio_
#         print(explained_variance)
        # self.pca(x_train, y_train)

        # training
        # self.clf = RandomForestClassifier(max_depth=25, n_jobs=100)
        # self.clf = AdaBoostClassifier()
#         self.clf = GradientBoostingClassifier()
#         self.clf = XGBClassifier()
#         self.clf.fit(x_train, y_train)

        self.clf = Sequential()
        # Adding the input layer and the first hidden layer
        self.clf.add(Dense(output_dim = 20, init = 'uniform', activation = 'relu', input_dim = len(x_train.columns)))
        # Adding the second hidden layer
        self.clf.add(Dense(output_dim = 10, init = 'uniform', activation = 'relu'))
        # Adding the output layer
        self.clf.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
        # Compiling Neural Network
        self.clf.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
        # Fitting our model 
        self.clf.fit(x_train, y_train, batch_size = 300, nb_epoch = 20)
        print('Training finished.')
        scores = cross_val_score(self.clf, x_train, y_train, cv=10, n_jobs=-1,scoring='accuracy')
        print(scores.mean())
        
#         scores = cross_val_score(self.clf, x_train, y_train, cv=10, n_jobs=-1)
#         print(scores.mean()) 


    # predicts the probablity of a machine affected by malware. The input test_data is a pandas dataframe.
    # This function must return the predicted probabilities for the test machines
    def predict_probablities(self, test_data):
        print('Predict malware')
        test_data = self.clean(test_data)
        test_data[test_data.columns] = self.scaler.fit_transform(test_data[test_data.columns])
        predictions = self.clf.predict(test_data)
        return predictions


# calls the train function and saves the model_file
# # donot modify the contents
if __name__ == "__main__":
    detectionModel = MalwareDetectionModel()
    detectionModel.train()
    filename = 'malwaremodel.pkl'
    f = open(filename, 'wb')
    dill.dump(detectionModel, f)
    f.close()
    
    
# dm = MalwareDetectionModel()

['IsBeta', 'PuaMode', 'Census_IsWIMBootEnabled', 'Census_ThresholdOptIn', 'Census_IsFlightingInternal', 'Census_IsFlightsDisabled', 'AutoSampleOptIn', 'SMode', 'Census_IsPortableOperatingSystem', 'Census_DeviceFamily', 'UacLuaenable', 'Census_IsVirtualDevice', 'ProductName', 'HasTpm', 'IsSxsPassiveMode', 'Firewall', 'AVProductsEnabled', 'RtpStateBitfield', 'OsVer', 'Platform', 'Census_IsPenCapable', 'IsProtected', 'Census_IsAlwaysOnAlwaysConnectedCapable', 'Census_FlightRing', 'Census_HasOpticalDiskDrive', 'Census_OSArchitecture', 'Processor', 'PuaMode', 'Census_ProcessorClass', 'DefaultBrowsersIdentifier', 'Census_IsFlightingInternal', 'Census_InternalBatteryType', 'Census_ThresholdOptIn', 'Census_IsWIMBootEnabled', 'SmartScreen', 'OrganizationIdentifier', 'SMode', 'Census_OSUILocaleIdentifier', 'MachineIdentifier']
Epoch 1/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.6931 - acc: 0.5049
Epoch 2/20
10000/10000 [==============================] - 20s 2ms/step 

TypeError: Cannot clone object '<keras.models.Sequential object at 0x13DDF250>' (type <class 'keras.models.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.